Лабораторная работа 4

Карягин Александр ИУ5-22

Задание:

*   Выбрать произвольный набор данных (датасет), предназначенный для построения рекомендательных моделей.
*   Опираясь на материалы лекции, сформировать рекомендации для одного пользователя (объекта) двумя произвольными способами.
*   Сравнить полученные рекомендации (если это возможно, то с применением метрик).


In [1]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple
from scipy import stats
from IPython.display import Image
from IPython.display import Image
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.datasets import load_iris, load_boston
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score 
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
# from matplotlib_venn import venn2
%matplotlib inline 
sns.set(style="ticks")

##Чтение и обработка данных

In [2]:
data = pd.read_csv('winemag.csv')
data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [3]:
data.shape

(129971, 14)

In [4]:
description_data = data[data['description'].notnull()]
description_data.shape

(129971, 14)

In [5]:
title = description_data['title'].values
title[0:5]

array(['Nicosia 2013 Vulkà Bianco  (Etna)',
       'Quinta dos Avidagos 2011 Avidagos Red (Douro)',
       'Rainstorm 2013 Pinot Gris (Willamette Valley)',
       'St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore)',
       "Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley)"],
      dtype=object)

In [6]:
descriptions = description_data['description'].values
descriptions[0:5]

array(["Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",
       "This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.",
       'Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.',
       'Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.',
       "Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it

In [7]:
description_data.keys()

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')

In [8]:
wine_ids = description_data['Unnamed: 0'].values
wine_ids

array([     0,      1,      2, ..., 129968, 129969, 129970])

In [9]:
%%time
tfidf = TfidfVectorizer()
description_matrix = tfidf.fit_transform(descriptions)
description_matrix

CPU times: user 4.42 s, sys: 94 ms, total: 4.51 s
Wall time: 4.53 s


In [10]:
description_matrix

<129971x31275 sparse matrix of type '<class 'numpy.float64'>'
	with 4475479 stored elements in Compressed Sparse Row format>

##Фильтрация на основе содержания. Метод k-ближайших соседей

In [11]:
class SimplerKnnRecomender:
  def __init__(self, X_matrix, X_ids, X_title, X_overview):
        """
        Входные параметры:
        X_matrix - обучающая выборка (матрица объект-признак)
        X_ids - массив идентификаторов объектов
        X_title - массив названий объектов
        X_overview - массив описаний объектов
        """
        #Сохраняем параметры в переменных объекта
        self._X_matrix = X_matrix
        self.df = pd.DataFrame(
            {'id': pd.Series(X_ids, dtype='int'),
            'title': pd.Series(X_title, dtype='str'),
            'overview': pd.Series(X_overview, dtype='str'),
            'dist': pd.Series([], dtype='float')})
  
  def recommend_for_single_object(self, K: int, \
                X_matrix_object, cos_flag = True, manh_flag = False):
        """
        Метод формирования рекомендаций для одного объекта.
        Входные параметры:
        K - количество рекомендуемых соседей 
        X_matrix_object - строка матрицы объект-признак, соответствующая объекту
        cos_flag - флаг вычисления косинусного расстояния
        manh_flag - флаг вычисления манхэттэнского расстояния
        Возвращаемое значение: K найденных соседей
        """
        
        scale = 1000000
        # Вычисляем косинусную близость
        if cos_flag:
            dist = cosine_similarity(self._X_matrix, X_matrix_object)
            self.df['dist'] = dist * scale
            res = self.df.sort_values(by='dist', ascending=False)
            # Не учитываем рекомендации с единичным расстоянием,
            # так как это искомый объект
            res = res[res['dist'] < scale]
        
        else:
            if manh_flag:
                dist = manhattan_distances(self._X_matrix, X_matrix_object)
            else:
                dist = euclidean_distances(self._X_matrix, X_matrix_object)
            self.df['dist'] = dist * scale
            res = self.df.sort_values(by='dist', ascending=True)
            # Не учитываем рекомендации с единичным расстоянием,
            # так как это искомый объект
            res = res[res['dist'] > 0.0]            
        
        # Оставляем К первых рекомендаций
        res = res.head(K)
        return res

In [12]:
test_id = 11
print(title[test_id])
print(descriptions[test_id])

Leon Beyer 2012 Gewurztraminer (Alsace)
This is a dry wine, very spicy, with a tight, taut texture and strongly mineral character layered with citrus as well as pepper. It's a food wine with its almost crisp aftertaste.


In [13]:
test_matrix = description_matrix[test_id]
test_matrix

<1x31275 sparse matrix of type '<class 'numpy.float64'>'
	with 25 stored elements in Compressed Sparse Row format>

In [14]:
skr1 = SimplerKnnRecomender(description_matrix, wine_ids, title, descriptions)

In [15]:
# 15 вин, наиболее похожих на Leon Beyer 2012 Gewurztraminer (Alsace)
# в порядке убывания схожести на основе косинусного сходства
rec1 = skr1.recommend_for_single_object(15, test_matrix)
rec1

,id,title,overview,dist
24045,24045,Domaine Michel Thomas et Fils 2015 Rosé (Sance...,The wine is textured and tight with crisp acid...,633624.990866
90700,90700,Henri de Villamont 2014 Morgeot Premier Cru (...,This wine is still tight and crisp. It has ple...,442624.176096
58330,58330,Schröder & Schÿler 2013 Chartron la Fleur (Bo...,"The wine is tight and nervy, very fresh, crisp...",432556.705703
66081,66081,Maison Champy 2014 Viré-Clessé,This taut and structured wine has weight as we...,430242.028148
78572,78572,Domaine Olivier Merlin 2014 Mâcon La Roche Vi...,"This wine is tight, structured and taut. Still...",428504.458538
105230,105230,Domaine Nigri 2013 Pierre de Lune (Jurançon Sec),This rich and ripe wine is full of apricot and...,425886.605501
25907,25907,Louis Max 2014 Mâcon-Villages,"Tight and structured, this wine has minerality...",424385.444731
99011,99011,Joseph Drouhin 2013 Les Clos (Macon-Bussières),This crisp wine offers plenty of acidity as we...,423757.525560
5406,5406,Aveleda 2015 Alvarinho (Vinho Verde),Ripe Alvarinho gives a wine that is rich as we...,421592.529700
22652,22652,Maison Malet Roquefort 2012 Léo de la Gaffeliè...,"Very herbaceous in character, this is a wine t...",418388.507228


In [16]:
# При поиске с помощью Евклидова расстояния получаем такой же результат
rec2 = skr1.recommend_for_single_object(15, test_matrix, cos_flag = False)
rec2

,id,title,overview,dist
24045,24045,Domaine Michel Thomas et Fils 2015 Rosé (Sance...,The wine is textured and tight with crisp acid...,8.560082e+05
90700,90700,Henri de Villamont 2014 Morgeot Premier Cru (...,This wine is still tight and crisp. It has ple...,1.055818e+06
58330,58330,Schröder & Schÿler 2013 Chartron la Fleur (Bo...,"The wine is tight and nervy, very fresh, crisp...",1.065311e+06
66081,66081,Maison Champy 2014 Viré-Clessé,This taut and structured wine has weight as we...,1.067481e+06
78572,78572,Domaine Olivier Merlin 2014 Mâcon La Roche Vi...,"This wine is tight, structured and taut. Still...",1.069108e+06
105230,105230,Domaine Nigri 2013 Pierre de Lune (Jurançon Sec),This rich and ripe wine is full of apricot and...,1.071553e+06
25907,25907,Louis Max 2014 Mâcon-Villages,"Tight and structured, this wine has minerality...",1.072953e+06
99011,99011,Joseph Drouhin 2013 Les Clos (Macon-Bussières),This crisp wine offers plenty of acidity as we...,1.073539e+06
5406,5406,Aveleda 2015 Alvarinho (Vinho Verde),Ripe Alvarinho gives a wine that is rich as we...,1.075553e+06
22652,22652,Maison Malet Roquefort 2012 Léo de la Gaffeliè...,"Very herbaceous in character, this is a wine t...",1.078528e+06


In [17]:
# Манхэттэнское расстояние дает несколько иные результаты поиска
rec3 = skr1.recommend_for_single_object(15, test_matrix, 
                                        cos_flag = False, manh_flag = True)
rec3

,id,title,overview,dist
24045,24045,Domaine Michel Thomas et Fils 2015 Rosé (Sance...,The wine is textured and tight with crisp acid...,3.865262e+06
22652,22652,Maison Malet Roquefort 2012 Léo de la Gaffeliè...,"Very herbaceous in character, this is a wine t...",5.251729e+06
35502,35502,Château de Piote 2012 Perles (Crémant de Bord...,"Tight and sharp, this is an herbaceous wine wi...",5.312967e+06
58330,58330,Schröder & Schÿler 2013 Chartron la Fleur (Bo...,"The wine is tight and nervy, very fresh, crisp...",5.316624e+06
25907,25907,Louis Max 2014 Mâcon-Villages,"Tight and structured, this wine has minerality...",5.354298e+06
21920,21920,Moncigale 2014 Frais et Délicat Rosé (Coteaux ...,"This is crisp, fruity with apple and citrus fl...",5.452536e+06
97201,97201,Ravoire et Fils 2013 Domaine la Rabiotte Rosé ...,"Tight, zingy and crisp, this wine has fresh, c...",5.535851e+06
70762,70762,Château du Seuil 2015 Domaine du Seuil (Borde...,The wine is tight and mineral in character. It...,5.564448e+06
128577,128577,Ravoire et Fils 2014 Domaine Bel Eouve Rosé (C...,"This is a tangy, spicy wine, a character that ...",5.628584e+06
78572,78572,Domaine Olivier Merlin 2014 Mâcon La Roche Vi...,"This wine is tight, structured and taut. Still...",5.644448e+06


##Коллаборативная фильтрация. Метод на основе сингулярного разложения

In [18]:
data.head(20)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
5,5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
6,6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,NaN,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo
7,7,France,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
8,8,Germany,Savory dried thyme notes accent sunnier flavor...,Shine,87,12.0,Rheinhessen,NaN,NaN,Anna Lee C. Iijima,NaN,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel
9,9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam


In [19]:
data3 = data[30000:55000]

In [20]:
# Количество уникальных дегустаторов
len(data3['taster_name'].unique())

20

In [21]:
# Количество уникальных вин
len(data3['title'].unique())

24517

In [22]:
# Сформируем матрицу взаимодействий на основе рейтингов
# Используется идея из статьи - https://towardsdatascience.com/beginners-guide-to-creating-an-svd-recommender-system-1fd7326d1f65
def create_utility_matrix(data):
    itemField = 'title'
    userField = 'taster_name'
    valueField = 'points'  
    
    userList = data[userField].tolist()
    itemList = data[itemField].tolist()
    valueList = data[valueField].tolist()    
    
    users = list(set(userList))
    items = list(set(itemList))    
    
    users_index = {users[i]: i for i in range(len(users))}    
    pd_dict = {item: [0.0 for i in range(len(users))] for item in items}    
    
    for i in range(0,data.shape[0]):
        item = itemList[i]
        user = userList[i]
        value = valueList[i]    
        pd_dict[item][users_index[user]] = value    
    
    X = pd.DataFrame(pd_dict)
    X.index = users
        
    itemcols = list(X.columns)
    items_index = {itemcols[i]: i for i in range(len(itemcols))}
    
    return X, users_index, items_index

In [23]:
%%time
user_item_matrix, users_index, items_index = create_utility_matrix(data3)

CPU times: user 959 ms, sys: 13.2 ms, total: 972 ms
Wall time: 978 ms


In [24]:
user_item_matrix

,Zull 2011 Lust & Laune Rosé (Niederösterreich),Massolino 2012 Parussi (Barolo),Arndorfer 2014 Strasser Weinberge Riesling (Niederösterreich),Lapostolle 2011 Canto de Apalta Red (Rapel Valley),Alvear 2005 Anada Pedro Ximénez (Montilla-Moriles),Veranda 2015 Pinot Noir (Bío Bío Valley),Equinox 1998 Blanc De Blanc Chardonnay (Santa Cruz Mountains),Dievole 2013 Chianti Classico,Domaine des Baumard 2012 Vert de l'Or Verdelho (Vin de Table Francais),Felton Road 2007 Cornish Point Pinot Noir (Central Otago),...,Inglenook 2014 Blancaneaux White (Rutherford),Château Cruzeau 2015 Saint-Émilion,Storrs 2009 Viento Vineyard Gewürztraminer (Monterey),Domaine de Courteillac 2009 Bordeaux Supérieur,Swanson 2008 Pinot Grigio (Napa Valley),Alex Gambal 2011 Puligny-Montrachet,Renwood 2012 Grandpere Special Reserve Zinfandel (Amador County),Château Sainte Barbe 2006 Merlot (Bordeaux),Maximin Grünhäuser 2015 Herrenberg Spätlese Grosse Lage Riesling (Mosel),Gunter Triebaumer 2007 Trie White (Burgenland)
NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,87.0,0.0,84.0,0.0,0.0,0.0,0.0,0.0
Joe Czerwinski,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fiona Adams,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alexander Peartree,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Michael Schachner,0.0,0.0,0.0,92.0,91.0,85.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kerin O’Keefe,0.0,94.0,0.0,0.0,0.0,0.0,0.0,88.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Anne Krebiehl MW,0.0,0.0,92.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Roger Voss,85.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90.0,0.0,...,0.0,90.0,0.0,88.0,0.0,92.0,0.0,84.0,0.0,87.0
Lauren Buzzeo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Susan Kostrzewa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
user_item_matrix.head(20)

,Zull 2011 Lust & Laune Rosé (Niederösterreich),Massolino 2012 Parussi (Barolo),Arndorfer 2014 Strasser Weinberge Riesling (Niederösterreich),Lapostolle 2011 Canto de Apalta Red (Rapel Valley),Alvear 2005 Anada Pedro Ximénez (Montilla-Moriles),Veranda 2015 Pinot Noir (Bío Bío Valley),Equinox 1998 Blanc De Blanc Chardonnay (Santa Cruz Mountains),Dievole 2013 Chianti Classico,Domaine des Baumard 2012 Vert de l'Or Verdelho (Vin de Table Francais),Felton Road 2007 Cornish Point Pinot Noir (Central Otago),...,Inglenook 2014 Blancaneaux White (Rutherford),Château Cruzeau 2015 Saint-Émilion,Storrs 2009 Viento Vineyard Gewürztraminer (Monterey),Domaine de Courteillac 2009 Bordeaux Supérieur,Swanson 2008 Pinot Grigio (Napa Valley),Alex Gambal 2011 Puligny-Montrachet,Renwood 2012 Grandpere Special Reserve Zinfandel (Amador County),Château Sainte Barbe 2006 Merlot (Bordeaux),Maximin Grünhäuser 2015 Herrenberg Spätlese Grosse Lage Riesling (Mosel),Gunter Triebaumer 2007 Trie White (Burgenland)
NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,87.0,0.0,84.0,0.0,0.0,0.0,0.0,0.0
Joe Czerwinski,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fiona Adams,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alexander Peartree,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Michael Schachner,0.0,0.0,0.0,92.0,91.0,85.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kerin O’Keefe,0.0,94.0,0.0,0.0,0.0,0.0,0.0,88.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Anne Krebiehl MW,0.0,0.0,92.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Roger Voss,85.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90.0,0.0,...,0.0,90.0,0.0,88.0,0.0,92.0,0.0,84.0,0.0,87.0
Lauren Buzzeo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Susan Kostrzewa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# Выделение тестовой строки
user_item_matrix__test = user_item_matrix.loc['Kerin O’Keefe']
user_item_matrix__test

Zull 2011 Lust & Laune Rosé (Niederösterreich)                               0.0
Massolino 2012 Parussi  (Barolo)                                            94.0
Arndorfer 2014 Strasser Weinberge Riesling (Niederösterreich)                0.0
Lapostolle 2011 Canto de Apalta Red (Rapel Valley)                           0.0
Alvear 2005 Anada Pedro Ximénez (Montilla-Moriles)                           0.0
                                                                            ... 
Alex Gambal 2011  Puligny-Montrachet                                         0.0
Renwood 2012 Grandpere Special Reserve Zinfandel (Amador County)             0.0
Château Sainte Barbe 2006 Merlot  (Bordeaux)                                 0.0
Maximin Grünhäuser 2015 Herrenberg Spätlese Grosse Lage Riesling (Mosel)     0.0
Gunter Triebaumer 2007 Trie White (Burgenland)                               0.0
Name: Kerin O’Keefe, Length: 24517, dtype: float64

In [27]:
#taster_names = description_data['taster_name'].unique()
taster_names = np.delete(data3['taster_name'].unique(), 0)
taster_names = np.delete(taster_names, 7)
taster_names

array(['Jim Gordon', 'Michael Schachner', 'Matt Kettmann',
       'Sean P. Sullivan', 'Roger Voss', 'Virginie Boone',
       'Joe Czerwinski', 'Paul Gregutt', 'Mike DeSimone', 'Jeff Jenssen',
       nan, 'Anna Lee C. Iijima', 'Susan Kostrzewa', 'Lauren Buzzeo',
       'Alexander Peartree', 'Fiona Adams', 'Carrie Dykes',
       'Christina Pickard'], dtype=object)

In [28]:
# Оставшаяся часть матрицы для обучения
user_item_matrix__train = user_item_matrix.loc[taster_names]
user_item_matrix__train

,Zull 2011 Lust & Laune Rosé (Niederösterreich),Massolino 2012 Parussi (Barolo),Arndorfer 2014 Strasser Weinberge Riesling (Niederösterreich),Lapostolle 2011 Canto de Apalta Red (Rapel Valley),Alvear 2005 Anada Pedro Ximénez (Montilla-Moriles),Veranda 2015 Pinot Noir (Bío Bío Valley),Equinox 1998 Blanc De Blanc Chardonnay (Santa Cruz Mountains),Dievole 2013 Chianti Classico,Domaine des Baumard 2012 Vert de l'Or Verdelho (Vin de Table Francais),Felton Road 2007 Cornish Point Pinot Noir (Central Otago),...,Inglenook 2014 Blancaneaux White (Rutherford),Château Cruzeau 2015 Saint-Émilion,Storrs 2009 Viento Vineyard Gewürztraminer (Monterey),Domaine de Courteillac 2009 Bordeaux Supérieur,Swanson 2008 Pinot Grigio (Napa Valley),Alex Gambal 2011 Puligny-Montrachet,Renwood 2012 Grandpere Special Reserve Zinfandel (Amador County),Château Sainte Barbe 2006 Merlot (Bordeaux),Maximin Grünhäuser 2015 Herrenberg Spätlese Grosse Lage Riesling (Mosel),Gunter Triebaumer 2007 Trie White (Burgenland)
Jim Gordon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,92.0,0.0,0.0,0.0
Michael Schachner,0.0,0.0,0.0,92.0,91.0,85.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Matt Kettmann,0.0,0.0,0.0,0.0,0.0,0.0,92.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sean P. Sullivan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Roger Voss,85.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90.0,0.0,...,0.0,90.0,0.0,88.0,0.0,92.0,0.0,84.0,0.0,87.0
Virginie Boone,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Joe Czerwinski,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Paul Gregutt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mike DeSimone,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jeff Jenssen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
%%time
U, S, VT = np.linalg.svd(user_item_matrix__train.T)
V = VT.T

CPU times: user 35.7 s, sys: 3.87 s, total: 39.6 s
Wall time: 25.1 s


In [30]:
# Матрица соотношения между дегустаторами и латентными факторами
U.shape

(24517, 24517)

In [31]:
# Матрица соотношения между объектами и латентными факторами
V.shape

(18, 18)

In [32]:
S.shape

(18,)

In [33]:
Sigma = np.diag(S)
Sigma.shape

(18, 18)

In [34]:
# Диагональная матрица сингулярных значений
Sigma

array([[6328.37615756,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        , 6214.00788753,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        , 4603.41568838,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , 3880.90866797,

In [35]:
# Используем 3 первых сингулярных значения
r=3
Ur = U[:, :r]
Sr = Sigma[:r, :r]
Vr = V[:, :r]
# Матрица соотношения между новым дегустатором и латентными факторами
test_user = np.mat(user_item_matrix__test.values)
test_user.shape, test_user

((1, 24517), matrix([[ 0., 94.,  0., ...,  0.,  0.,  0.]]))

In [36]:
tmp = test_user * Ur * np.linalg.inv(Sr)
tmp

matrix([[ 3.78394162e-04, -4.35827216e-06,  2.92221681e-18]])

In [37]:
test_user_result = np.array([tmp[0,0], tmp[0,1], tmp[0,2]])
test_user_result

array([ 3.78394162e-04, -4.35827216e-06,  2.92221681e-18])

In [38]:
# Вычисляем косинусную близость между текущим дегустатором 
# и остальными дегустаторами
cos_sim = cosine_similarity(Vr, test_user_result.reshape(1, -1))
cos_sim[:10]

array([[ 9.99999728e-01],
       [-1.53344225e-18],
       [-1.11037310e-33],
       [ 4.88453854e-36],
       [-4.12491330e-04],
       [ 9.99999975e-01],
       [ 0.00000000e+00],
       [-1.04994959e-03],
       [ 0.00000000e+00],
       [ 0.00000000e+00]])

In [39]:
# Преобразуем размерность массива
cos_sim_list = cos_sim.reshape(-1, cos_sim.shape[0])[0]
cos_sim_list[:10]

array([ 9.99999728e-01, -1.53344225e-18, -1.11037310e-33,  4.88453854e-36,
       -4.12491330e-04,  9.99999975e-01,  0.00000000e+00, -1.04994959e-03,
        0.00000000e+00,  0.00000000e+00])

In [40]:
# Находим наиболее близкого дегустатора
recommended_user_id = np.argsort(-cos_sim_list)[0]
recommended_user_id

5

In [41]:
test_user

matrix([[ 0., 94.,  0., ...,  0.,  0.,  0.]])

In [42]:
# Получение названия вина
wine_list = list(user_item_matrix.columns)
def film_name_by_movieid(ind):
    try:
        wine = wine_list[ind]
        #print(wineId)
        #flt_links = data3[data['movieId'] == wineId]
        #tmdbId = int(flt_links['tmdbId'].values[0])
        #md_links = df_md[df_md['id'] == tmdbId]
        #res = md_links['title'].values[0]
        return wine
    except:
        return '' 

In [43]:
# Вина, которые оценивал текущий дегустатор:
i=1
for idx, item in enumerate(np.ndarray.flatten(np.array(test_user))):
    if item > 0:
        film_title = film_name_by_movieid(idx)
        print('{} - {} - {}'.format(idx, film_title, item))
        if i==20:
            break
        else:
            i+=1

1 - Massolino 2012 Parussi  (Barolo) - 94.0
7 - Dievole 2013  Chianti Classico - 88.0
14 - Bonacchi 2014 Il Selciato  (Rosso di Montalcino) - 85.0
27 - Medici Ermete 2016 Solo Red - 88.0
29 - Castelli Martinozzi 2012  Brunello di Montalcino - 92.0
53 - Maso Martis 2004 Madame Martis Brut Riserva Sparkling (Trento) - 88.0
65 - Lovo 2016  Prosecco - 85.0
83 - Cordero di Montezemolo 2012 Funtanì  (Barbera d'Alba Superiore) - 89.0
102 - Fattori 2012 Danieli  (Soave) - 87.0
132 - Canella 2016 Extra Dry  (Valdobbiadene Prosecco Superiore) - 85.0
134 - Piazzo Armando 2012  Barbaresco - 89.0
135 - Feudi di San Gregorio 2011  Taurasi - 91.0
140 - Coppiere 2013  Chianti Classico - 86.0
144 - Giuseppe Cortese 2012 Rabajà  (Barbaresco) - 92.0
166 - Perla del Garda 2013 Madonna della Scoperta Riserva  (Lugana) - 86.0
208 - La Madonnina 2011 Bello Stento  (Chianti Classico) - 87.0
216 - Marchesi Antinori 2013 Pèppoli  (Chianti Classico) - 89.0
247 - Nobilissima 2015  Chianti - 87.0
262 - Feudo Solar

In [44]:
# Вина, которые оценивал наиболее схожий дегустатор:
i=1
recommended_user_item_matrix = user_item_matrix.loc[['Roger Voss']]
for idx, item in enumerate(np.ndarray.flatten(np.array(recommended_user_item_matrix))):
    if item > 0:
        film_title = film_name_by_movieid(idx)
        print('{} - {} - {}'.format(idx, film_title, item))
        if i==20:
            break
        else:
            i+=1

0 - Zull 2011 Lust & Laune Rosé (Niederösterreich) - 85.0
8 - Domaine des Baumard 2012 Vert de l'Or Verdelho (Vin de Table Francais) - 90.0
11 - Château Bélingard 2015 Bélingard  (Bergerac Rosé) - 85.0
18 - Louis Latour 2005 Domaine Latour  (Aloxe-Corton) - 89.0
20 - Casca Wines 2009 Cape Roca Fisherman Rosé (Tejo) - 85.0
33 - Deutz 2004 Brut  (Champagne) - 90.0
36 - Boeckel 2012 Zotzenberg Grand Cru Riesling (Alsace) - 93.0
47 - Château Haut-Rian 2016  Bordeaux Blanc - 87.0
48 - Château Trottevieille 2007  Saint-Émilion - 88.0
51 - Le Bonheur 2015 Rosé (Côtes de Provence) - 86.0
59 - Brandl 2011 Heiligenstein Erste Lage Reserve Riesling (Kamptal) - 90.0
64 - Domaine Reverdy-Ducroux 2014 Chêne Marchand  (Sancerre) - 88.0
70 - Marc Kreydenweiss 2010 Kritt Pinot Blanc (Alsace) - 91.0
72 - Markus Huber 2010 Berg Reserve Grüner Veltliner (Traisental) - 93.0
73 - Jean Laurent NV Fleur de Celles Brut  (Champagne) - 93.0
76 - Maison Champy 2010 Les Chênevottes Premier Cru  (Chassagne-Montrach

####Как видно, фильтрация на основе содержания и коллаборативная фильтрация показывают различные результаты работы в рамках рекомендательных систем